<a href="https://colab.research.google.com/github/yakaboskic/ENGS_108_Fall_2020/blob/master/solutions/assign_6_ENGS_108_Fall_2020_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2020 Assignment 6**

*Due November 6, 2020 at 11:59PM on Canvas*

**Instructors:** George Cybenko

**TAs:** Chase Yakaboski


---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in 
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---

In [ ]:
''' Import Statements '''
from collections import defaultdict
import copy
import itertools

import random
# Don't mess with this (gives reproducible results)
random.seed(444)

## **Problem 1: Reinforcement Learning**
In this problem we will play a game of cops and robbers. The game is played on a fixed undirected, simple, and finite graph $G$. There are two players, a cop and a robber. It is the goal of the cop to catch the robber in as few moves as possible. 

The graph $G$ has the following properties:
  - It contains a single $n$ node cycle and random additional edges to make the graph connected.

The game starts, with the cop taking their choice of vertex in $G$ and then the robber selects a random vertex in $G$ that is not occupied by the cop. At every point in the game both players know the positions of each other, and in this version of the problem we will say that the robber is drunk (i.e. they will randomly choose there next that instead of employing a policy).

The availabe actions of the cop and associated reward function is:
  - Move to a node not connected to their present node (and the cop stays in the current position): -5.
  - Move to an adjacent node (including staying at current node): -1.
  - Move to the node occupied by robber: +100.
>
> **Part 1** Building a Graph Class.
>> **(a)** Using the provided skeleton build a general graph class for this problem for $n$ nodes. You are expected to implement *add_edge*, *make_random_graph*, *check_connected*.

In [ ]:
class Graph:
  """ Our graph class.
  Args:
    n: Number of nodes in our cycle.
    m: Total number of nodes in graph, where n >= m.
  """
  def __init__(self, n, m):
    self.n = n
    self.m = m
    # A default dict is just a dict that won't raise a KeyError, it instead fills
    # the unknown key with a default, in our case an empty list.
    self.G = defaultdict(list)
    
  def add_edge(self, u, v):
    """
    Make a function that will add an edge to the graph.
    """
    self.G[u].append(v)
    self.G[v].append(u)


  def make_random_graph(self):
    """ First make a cycle of given length and then add random additional edges
    in such a way that the final graph will be connected.
    """
    # Make n length cycle
    for u in range(self.n):
      v = u + 1
      if v == self.n:
        v = 0
      self.add_edge(u, v)
    
    # Add additional nodes until random graph is connected
    while True:
      _graph = copy.copy(self)
      #-- Add random nodes
      for u in range(self.n, self.m):
        v = random.choice(list(_graph.G.keys()))
        _graph.add_edge(u, v)
      if self.check_connected(_graph.G):
        self.G = _graph.G
        return True

  def check_connected(self, G):
    """ Perform a Depth First Search.
    """
    print(G)
    start_node = random.choice(list(G.keys()))
    return dfs(G, {}, start_node, start_node)

def dfs(G, visited, u, parent):
  """ Implement your own depth first search. Many online resources for this.
  Args:
    G: Is the graph (adjency matrix) we want to test.
    visited: Is a dictionary that keeps track of the nodes you've visited.
    u: a starting node.
    parent: The parent node for DFS so that you can cancel recursion if you complete the cycle.
  """
  return True
  print('u', u)
  print('parent', parent)
  input()
  if u not in visited:
    visited[u] = True
    # If you is not a source node in the adjacency matrix.
    if u not in G:
      return False
  for v in G[u]:
    if v not in visited:
      if dfs(G, visited, v, parent):
        return True
    elif parent != v:
        return True
  return False

>> **(b)** Test out your graph class with a cops and robbers graph of $n=5$ and $m=10$.

In [ ]:
# Test
graph = Graph(5, 10)
graph.make_random_graph()
#dfs(graph.G, {}, 0, 0)

defaultdict(<class 'list'>, {0: [1, 4, 8], 1: [0, 2], 2: [1, 3, 6, 7, 9], 3: [2, 4, 5], 4: [3, 0], 5: [3], 6: [2], 7: [2], 8: [0], 9: [2]})


True

> **Part 2** Understanding the state space, i.e. the Game.
>> **(a)** Given the graph class you've created in Part 1. Develop a Cops and Robbers game class. Use the skeleton below to implement the following functions first: *get_successors*, *terminal_test*, *result*. 


In [ ]:
class CopsAndRobbers:
  def __init__(self, graph, start_state=None, rewards_table=None):
    """ This is the cops and robbers game class.
    Args:
      start_state: The starting state of the cop position and robber position in
        the graph. Should be a tuple of form (cop_pos, rob_pos). If None, choose random start state.
    """
    self.graph = graph
    self.G = graph.G

    if start_state is None:
      while True:
        cop_state = random.randint(0, self.graph.m - 1)
        rob_state = random.randint(0, self.graph.m - 1)
        if cop_state != rob_state:
          break
      self.state = (cop_state, rob_state)
    else:
      self.state = start_state
    self.rewards_table = rewards_table

  def terminal_test(self, state):
    if state[0] == state[1]:
      return True
    return False

  def get_successors(self, state):
    """ Return a list of successor states that can be reached from the current state.
    Hint: Remember only the cop can choose their action.
    """
    return [(u, state[1]) for u in range(self.graph.m)]

  def result(self, state, next_state):
    """ This function should return the state after the cop has made their move,
    and the drunk robber has moved accordingly.
    Args:
      state: Current state of (cop, rob).
      next_state: The state after the cop has moved (next_cop, rob). Calculated from get_successors.
    """
    # Robber has equal probability of moving to any adjacent state.
    next_cop_pos = next_state[0]
    reward = self.utility(state, next_state[0])
    if next_cop_pos not in self.graph.G[state[0]]:
      next_cop_pos = state[0]
    # State after cop moves or doesn't
    next_state = (next_cop_pos, next_state[1])
    if self.terminal_test(next_state):
      return True, reward
    rob_pos = random.choice(self.graph.G[state[1]])
    self.state = (next_cop_pos, rob_pos)
    return False, reward

  def utility(self, state, action):
    return self.rewards_table[(state, action)]

>> **(b)** In reinforcement learning we are often interested in calculating a rewards table that has possible states as its rows and possible actions as its columns and filled in with the associated reward given the Q(state, action) pair. Calculate the rewards table for any given graph. *Hint: This should be an $m^2$ x $m$ matrix.*  

In [ ]:
def calculateRewardsTable(graph):
  """ Make a rewards table dictionary of the from table[(state, action)] = reward.
  """
  possible_states = [prod for prod in itertools.product(range(graph.m), repeat=2)]
  actions = [i for i in range(graph.m)]
  table = {}
  for state in possible_states:
    for action in actions:
      if action in graph.G[state[0]]:
        if action == state[1]:
          table[(state, action)] = 100
        else:
          table[(state, action)] = -1
      else:
        if action == state[0]:
          if state[0] == state[1]:
            table[(state, action)] = 100
          else:
            table[(state, action)] = -1
        else:
          table[(state, action)] = -5
  return table

>> **(c)** Now that we have our reward table, try to solve the problem in a brute-force manner (without reinforcement learning). I.e. try to reach the terminal state, or find get a reward of 100. 

In [ ]:
epochs = 0
rewards = []
penalties = 0
graph = Graph(5, 10)
graph.make_random_graph()
rewards_table = calculateRewardsTable(graph)
game = CopsAndRobbers(graph, (0,1), rewards_table=rewards_table)

# Simulation loop
while True:
  next_state = random.choice(game.get_successors(game.state))
  terminal, reward = game.result(game.state, next_state)
  rewards.append(reward)
  if terminal:
    break
  else:
    if reward < 0:
      penalties += 1
  epochs += 1

defaultdict(<class 'list'>, {0: [1, 4], 1: [0, 2, 7], 2: [1, 3, 5, 6], 3: [2, 4], 4: [3, 0, 9], 5: [2], 6: [2], 7: [1, 8], 8: [7], 9: [4]})


> **Part 3** Q-learning. 
>
> Up to this point we have build a graph class, built a game class, ran a brute-force simulation of an agent traversing the space randomly, and now we will dive into Q-learning in the hopes of maximizing the rewards and efficiency of capturing the drunk robber. 
>
>> **(a)** Using the skeleton below, implement a training loop to learn a Q-table for a given graph and game. 

In [ ]:
def init_q_table(graph):
  possible_states = [prod for prod in itertools.product(range(graph.m), repeat=2)]
  actions = [i for i in range(graph.m)]
  q_table = {state: {} for state in possible_states}
  for state in possible_states:
    for action in actions:
      q_table[state][action] = 0
  return q_table

In [ ]:
import operator
def get_best_q_action(action_dict):
  return max(action_dict.items(), key=operator.itemgetter(1))[0]

In [ ]:
graph = Graph(5, 10)
graph.make_random_graph()
rewards_table = calculateRewardsTable(graph)
q_table = init_q_table(graph)

# Hyperparameters
alpha = 0.1
gamma = 1
epsilon = 0.1

for i in range(1, 2001):
  game = CopsAndRobbers(graph, None, rewards_table=rewards_table)
  epochs = 0
  rewards = []
  penalties = 0

  while True:
    # Make a copy of game state
    state = copy.copy(game.state)
    if random.uniform(0,1) < epsilon:
      next_state = random.choice(game.get_successors(state))
    else:
      next_cop_pos = get_best_q_action(q_table[state])
      next_state = (next_cop_pos, state[1])
    
    # Set new game state
    terminal, reward = game.result(game.state, next_state)
    rewards.append(reward)

    # Get old q table value. Remember it's just the cops action
    old_q = q_table[state][next_state[0]]

    # Get next state max q
    next_max = max(q_table[next_state].items(), key=operator.itemgetter(1))[1]

    # Calculate new value
    new_value = (1 - alpha) * old_q + alpha * (reward + gamma * next_max)
    q_table[state][next_state[0]] = new_value

    if terminal:
      break

    # Record info
    if reward < 0:
      penalties += 1
    epochs += 1

  if i % 100 == 0:
      print(f"Episode: {i}")

defaultdict(<class 'list'>, {0: [1, 4, 5], 1: [0, 2, 6, 7, 8], 2: [1, 3], 3: [2, 4], 4: [3, 0], 5: [0, 9], 6: [1], 7: [1], 8: [1], 9: [5]})
Episode: 100
Episode: 200
Episode: 300
Episode: 400
Episode: 500
Episode: 600
Episode: 700
Episode: 800
Episode: 900
Episode: 1000
Episode: 1100
Episode: 1200
Episode: 1300
Episode: 1400
Episode: 1500
Episode: 1600
Episode: 1700
Episode: 1800
Episode: 1900
Episode: 2000


In [ ]:
q_table

{(0, 0): {0: 3370.0,
  1: 456.66478392710104,
  2: 193.60857126157532,
  3: 181.01284889994392,
  4: 574.1151798904795,
  5: 1095.3315516603584,
  6: 310.1753383902017,
  7: 379.9022515330603,
  8: 541.236220106021,
  9: 554.689189852035},
 (0, 1): {0: 623.6598664343314,
  1: 2163.389773011825,
  2: 509.7988375506296,
  3: 275.8167129569913,
  4: 785.8424188311781,
  5: 449.4304867875878,
  6: 109.97508940954154,
  7: 326.30435570176604,
  8: 303.911407676137,
  9: 500.83004458428354},
 (0, 2): {0: -0.2,
  1: 227.8990509384583,
  2: 9.50343446391116,
  3: 9.721768796230904,
  4: -0.1,
  5: 45.76778821208424,
  6: 31.545822699027,
  7: 33.53436591481071,
  8: 12.934515723313385,
  9: -0.51},
 (0, 3): {0: -0.2,
  1: 218.0067574020644,
  2: 14.199652063559224,
  3: -0.5,
  4: -0.1,
  5: 31.702613789262987,
  6: -0.5,
  7: -0.5,
  8: -0.5,
  9: -0.519},
 (0, 4): {0: -0.1,
  1: 3.473554203679166,
  2: -0.5,
  3: -0.5,
  4: 158.8445830612053,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0},
 (0, 5):

>> **(b)** Evalute your new Q-learning agent over a 100 epochs, by choosing your actions based on the argmax of the Q-table caluculated in (a) and report the average number of penalities, average time, and average number of steps it took to find the robber with your new Q-learning strategy. 

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 10

print(graph.G)

for i in range(episodes):
    game = CopsAndRobbers(graph, None, rewards_table=rewards_table)
    epochs, penalties, reward = 0, 0, 0
    print(f"Episode: {i}")
    while True:
        state = copy.copy(game.state)
        action = get_best_q_action(q_table[state])
        next_state = (action, state[1])
        
        # Set new game state
        terminal, reward = game.result(game.state, next_state)
        rewards.append(reward)
  
        #print('State: {}\nActions: {}\nAction: {}\nProposed' \
        # 'Next State: {}\nReward: {}\n Next State: {}\nTerminal: {}'.format(state, q_table[state], action, next_state, reward, game.state, terminal))
        #input()
        if reward < 0:
            penalties += 1
        epochs += 1

        if terminal:
          break

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

defaultdict(<class 'list'>, {0: [1, 4, 5], 1: [0, 2, 6, 7, 8], 2: [1, 3], 3: [2, 4], 4: [3, 0], 5: [0, 9], 6: [1], 7: [1], 8: [1], 9: [5]})
Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
Episode: 8
Episode: 9
Results after 10 episodes:
Average timesteps per episode: 5.0
Average penalties per episode: 4.0


>> **(c)** Compare your results with the brute-force method used in Part 2 and comment on the improvement. For instance, try varying graph configurations and look for any signs of improvement in certian instances. 